In [7]:
#import scikit learn decision.tree
import numpy as np
import pandas as pd
from sklearn import tree
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt



class MDL_weighted_randomforest():
  
    def __init__(self, n_estimators=100, max_features=5, min_samples_split=2,
                 min_gain=0, max_depth=float("inf")):
        self.n_estimators = n_estimators    # Number of trees
        self.max_features = max_features   #ideally, sqrt(# variables)
        self.min_samples_split = min_samples_split
        self.min_gain = min_gain           
        self.max_depth = max_depth         
        self.progressbar = progressbar.ProgressBar(widgets=bar_widgets)
        self.trees = []
        self.features = []
        self.lengths = [] #tree depth
        self.treefits = [] #R^2 for tree fits on train data
        for _ in range(n_estimators):
            self.trees.append(tree.DecisionTreeClassifier())
                    

    def fit(self, X, y):
                
        subsets = get_random_subsets(X, y, self.n_estimators)

        for i in range(self.n_estimators):
            X_subset, y_subset = subsets[i]
            #random subsets
            idx = np.random.choice(range(n_features), size=self.max_features, replace=True)
            #save indices for prediction
            self.features[i] = idx
            X_subset = X_subset[:, idx]
            self.trees[i].fit(X_subset, y_subset)
            # get tree length, then tree fit on train data
            self.lengths[i] = self.trees[i].decision_path(X_subset,Y_subset)
            self.treefits[i] = self.trees[i].score(X_subset, Y_subset)
                
    def predict(self, X, depth_weight, accuracy_weight):   #use cross validation on train/test splits to find optimal values for weights.
        y_preds = np.empty((X.shape[0], len(self.trees)))     #use validation data for final results
        
        for i, tree in enumerate(self.trees):
          
            idx = self.features[i]
            prediction = tree.predict(X[:, idx])
               #weighted prediction of these features 
            weighted_prediction = ((1 / depth_weight * self.lengths[i]) * prediction) + (accuracy_weight * prediction * self.treefits[i])
            
            y_preds[:, i] = weighted_prediction
            
        y_pred = []
        
        for weighted_prediction in y_preds:
            
            #rescale, so all votes are between 0,1 range
        
            normalized_prediction = min_max_scaler.fit_transform(weighted_prediction)
            estimate = np.mean(normalized_prediction)
            if estimate < .5:
                outcome = 0
            else:
                outcome = 1
            #if mean <.5, 0, else 1
            y_pred.append(np.bincount(outcome))
        return y_pred